# Value iteration
Value iteration works similar to policy evaluation. Instead of the expectation at each step we are taking the max. In policy evaluation we averaged over all successor states together with the immediate reward. Now we update the value function according to the maximizing successor state $s^{'}$ (Discounted by $\gamma$). The (deterministic) policy now simply chooses the action that maximizes the value. 

<img src="value_iter_pc.png" width="500"/>

In [199]:
# Imports
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [200]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [201]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """

    V = np.zeros(env.nS)
    v = np.zeros(env.nS)
    delta = np.zeros(env.nS)
    
    while True:
        policy = np.zeros([env.nS, env.nA])
        for s in range(env.nS):
            v[s] = V[s]
            
            # Rewards and best actions given the state
            rewards = np.asarray([env.P[s][a][0][2] for a in range(env.nA)])
            next_states = np.asarray([env.P[s][a][0][1] for a in range(env.nA)])
            best_action = np.argmax(rewards[:,None]+discount_factor*V[next_states][:,None])
            
            # Updating the value function 
            V[s] =  np.max(rewards[:,None]+discount_factor*V[next_states][:,None])
            # Updating the policy according to the best action
            policy[s,:] = np.eye(env.nA)[best_action,:] 
            
        # Break if 'finished'
        delta = np.abs(v-V)
        if delta.any() < theta:
            break
    return policy, V

In [202]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [203]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)